<a href="https://colab.research.google.com/github/KHANmdAFFAN/Data_science/blob/main/spotify_recomendation_system_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Description 

##Spotify Playlist Recommendation System

### Description of the Project
The goal of the challenge is to develop a system for the task of automatic playlist continuation. given a set of playlist features, participants' systems shall generate a list of recommended tracks that can be added to that playlist, thereby "continuing" the playlist ```


### Overall flow of the project
the data is structured in arrays that typically contain 1,000 playlists. each playlist is a dictionary that contains the following fields:[]




 * pid - integer - playlist id - the mpd id of this playlist. this is an integer between 0 and 999,999.
 
 * name - string - the name of the playlist
 
 * description - optional string - if present, the description given to the playlist. note that user-provided playlist descriptions are a relatively new feature of spotify, so most playlists do not have descriptions.
 
 * modified_at - seconds - timestamp (in seconds since the epoch) when this playlist was last updated. times are rounded to midnight gmt of the date when the playlist was last updated.
 
 * num_artists - the total number of unique artists for the tracks in the playlist.
 
 * num_albums - the number of unique albums for the tracks in the playlist 
 
 *num_tracks - the number of tracks in the playlist
 
 * num_followers - the number of followers this playlist had at the time the mpd was created. (note that the follower count does not including the playlist creator) 
 
 *num_edits - the number of separate editing sessions.Tracks added in a two hour window are considered to be added in a single editing session.

 *duration_ms - the total duration of all the tracks in the playlist (in milliseconds) 
 
 *collaborative - boolean - if true, the playlist is a collaborative playlist. multiple users may contribute tracks to a collaborative playlist.
 
 * tracks - an array of information about each track in the playlist. each element in the array is a dictionary with the following fields:

In [ ]:
#* track_name - the name of the track 
#* track_uri - the spotify uri of the track
#* album_name - the name of the track's album 
#* album_uri - the spotify uri of the album 
#* artist_name - the name of the track's primary artist 
#* artist_uri - the spotify uri of track's primary artist 
#* duration_ms - the duration of the track in milliseconds 
#* pos - the position of the track in the playlist (zero-based)

In [18]:
import pandas as pd 
import numpy as np
import json as json
import re


## **Data Prepration Phase**

In this phase we needed to flatten our data

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [14]:
data1 = json.load(open('/content/drive/MyDrive/Spotify_Playlist_Datasets.rar (Unzipped Files)/mpd.slice.0-999.json'))
data2 = json.load(open('/content/drive/MyDrive/Spotify_Playlist_Datasets.rar (Unzipped Files)/mpd.slice.1000-1999.json'))
data3 = json.load(open('/content/drive/MyDrive/Spotify_Playlist_Datasets.rar (Unzipped Files)/mpd.slice.2000-2999.json'))
data4 = json.load(open('/content/drive/MyDrive/Spotify_Playlist_Datasets.rar (Unzipped Files)/mpd.slice.3000-3999.json'))

df1= pd.DataFrame(data1['playlists'])
df2= pd.DataFrame(data2['playlists'])
df3= pd.DataFrame(data3['playlists'])
df4= pd.DataFrame(data4['playlists'])

df = pd.concat([df1,df2,df3,df4])

df= df.explode('tracks')
df= pd.concat([df.drop(['tracks'], axis=1), df['tracks'].apply(pd.Series)],axis=1)

# renaming the columns with the same name 
df.columns=['name', 'collaborative', 'pid', 'modified_at', 'num_tracks', 'num_albums', 'num_followers', 'num_edits', 'playlist_duration_ms', 'num_artists', 'description', 'pos', 'artist_name', 'track_uri', 'artist_uri', 'track_name', 'album_uri', 'track_duration_ms', 'album_name']

And we have data ready to manipulate it :


## the current features are not enough to analyze the tracks so it's needed to extract more features from tracks uri

### from above there are some features have uri in names , after searching i found this:

A spotify uri (uniform resource indicator) is a link that you can find in the share menu of any track, album, or artist profile on spotify.

When you click a link that consists of a spotify uri (rather than an http address), you're taken directly to the spotify application, without having to go through the web page first.


## it could be used and extract more data by api spotipy ,but i already have the name of the track , artist and album in the dataset so i'm gonna take a copy of dataframe and remove the uri features before making any process

In [19]:
#extract uri 
df['track_uri']= df["track_uri"].apply(lambda x: re.findall(r'\w+$', str(x))[0])